# Tutorial for Simple Route Optimization with ASKCOS 
Assumes ASKCOS exists in directory and can be imported. 
Follow installation instructions in README

### Import Targets and Scores

In [1]:
import pandas as pd 

target_csv = "target_list_7mols.csv"
df = pd.read_csv(target_csv)
target_dict = {smi: score for smi, score in zip(df['SMILES'], df['score'])}

### Build Tree 

In [ ]:
import sys 
sys.path.append('/home/jfromer/sparrow/askcos-core') # change this for your system to where askcos folder is located

from sparrow.tree_build_utils import build_rxn_graph
route_graph = build_rxn_graph(
    target_smis=list(target_dict.keys()),
    n_cpus=5,
    time_per_target=15
)

In [3]:
import pickle 
with open("paths.pickle", "rb") as f:
    paths = pickle.load(f)

In [5]:
paths

[]

Save the route_graph

In [3]:
route_graph.save(filename="simple_route_graph_w_scores.pickle")

### Set up the problem and solve 
First, define RouteSelector object and calculate reaction scores. Only reason to _not_ calculate scores is if you are loading a previously saved route_graph which has reaction scores already. (Reaction scores indicate how confident the ASCKOS forward predictor module is in desired product actually being produced)
Second, define optimization problem variables, set the objectives and constraints, and finally solve

In [ ]:
from sparrow.route_selector import RouteSelector

route_selector = RouteSelector(
    route_graph,
    target_dict,
    calc_reaction_scores=True,
)

route_selector.define_variables()
route_selector.set_objective()
route_selector.set_constraints()
route_selector.optimize()



### Visualize the results 

In [ ]:
from sparrow.visualizer import Visualizer
import matplotlib.pyplot as plt

vis = Visualizer(
    route_graph,
    nonzero_vars=route_selector.optimal_variables(),
    )

fig_out_dir = "tutorials/askcos_simple.png"
vis.plot_graph(fig_out_dir)

plt.show()